#Make a POST request

In [ ]:
import requests
url = "https://29nr10lpth.execute-api.us-east-1.amazonaws.com/skanner_predict_picture"
#url = "http://e7a41f244533.ngrok.io"

#arr = imread('aamrmacbsylkbahdhvcfmeqvgyjopc.jpg')
arr = imread(path_image)
data = {'media': arr.tolist()}
response = requests.post(url, json=data)
response

<Response [500]>

#Model

##Mnist

In [ ]:
import time
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import random

def resize100(img):
    return resize(img, (224,224), preserve_range=True, mode='reflect', anti_aliasing=True)

image = np.expand_dims(resize100(imread(path_image)),0)
image.shape

(1, 224, 224, 3)

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

def create_model():
  model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(2, activation='softmax')
  ])

  model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
  )

  return model

tfds.disable_progress_bar()
tf.enable_v2_behavior()
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

model = create_model()

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")


model.save("model")
model = tf.keras.models.load_model('model')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)


print("Saved model to disk")
# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights("model.h5")

# Evaluate the model from model.h5
arr = imread('aamrmacbsylkbahdhvcfmeqvgyjopc.jpg')
print(arr.shape)
image = resize100(imread('/content/aamrmacbsylkbahdhvcfmeqvgyjopc.jpg'))
print(image.shape)
image = normalize_img(image, 1)[0]
print(image.shape)
image =tf.image.rgb_to_grayscale(image, name=None)
print(image.shape)
t = model.predict(np.expand_dims(normalize_img(image,0)[0],0))
print(np.argmax(t))
print(t)

##Our Model 

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

def create_model():
    IMAGE_SIZE= [224,224]
    pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    pretrained_model.trainable = False 


    output = tf.keras.layers.Flatten()(pretrained_model.output)
    output = tf.keras.layers.Dense(units = 256, activation='relu',)(output) 
    output = tf.keras.layers.BatchNormalization()(output)
    output = tf.keras.layers.Dropout(0.9)(output)
    output = tf.keras.layers.Dense(2 ,activation='softmax')(output)

    model_vgg16 = tf.keras.Model(pretrained_model.input, output)

    opt = tf.keras.optimizers.Adam(learning_rate=0.1)
    #model.compile(optimizer=opt, loss= tf.keras.losses.BinaryCrossentropy() , metrics=['accuracy'])
    model_vgg16.compile(#loss='sparse_categorical_crossentropy',
                  loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', tf.keras.metrics.AUC()])
                  #metrics=['accuracy', tf.keras.metrics.AUC()])

    return model_vgg16

tfds.disable_progress_bar()
tf.enable_v2_behavior()
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

model = create_model()

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

In [ ]:
model = tf.keras.models.load_model('gdrive/My Drive/Thesis/models/model_aws.h5')

###Evaluations

In [ ]:
from PIL import Image
for i in range(1,10):
  arr = imread(f"gdrive/My Drive/Thesis/models/melanoma_{i}.jpg") / 255

  print(arr.shape)
  image = tf.image.resize(arr,[224,224])
  print(image.shape)
  t = model.predict(np.expand_dims(image,0))
  print(np.argmax(t))
  print(t)

(1705, 2450, 3)
(224, 224, 3)
0
[[0.9948632  0.00513678]]
(1705, 2450, 3)
(224, 224, 3)
0
[[0.9948632  0.00513678]]
(1705, 2450, 3)
(224, 224, 3)
0
[[0.9948632  0.00513678]]
(1705, 2450, 3)
(224, 224, 3)
0
[[0.9948632  0.00513678]]
(1705, 2450, 3)
(224, 224, 3)
0
[[0.9948632  0.00513678]]
(1705, 2450, 3)
(224, 224, 3)
0
[[0.9948632  0.00513678]]
(1705, 2450, 3)
(224, 224, 3)
0
[[0.9948632  0.00513678]]
(1705, 2450, 3)
(224, 224, 3)
0
[[0.9948632  0.00513678]]
(1705, 2450, 3)
(224, 224, 3)
0
[[0.9948632  0.00513678]]


In [ ]:
import tensorflow.compat.v2 as tf
import numpy as np
model1 = tf.keras.models.load_model('model_aws.h5')
model1.save_weights("weights_only.h5")
json_config = model1.to_json()
with open('model_config.json', 'w') as json_file:
    json_file.write(json_config)

##TF 1.14
with open('model_config.json') as json_file:
    json_config = json_file.read()
model2 = tf.keras.models.model_from_json(json_config)
model2.load_weights('weights_only.h5')


t = ["/content/benign_3.jpg", "/content/benign_5.jpg", "/content/melanoma_7.jpg", "/content/melanoma_4.jpg", "/content/melanoma.jpeg"]
model3 = tf.keras.models.load_model('/content/oversampling.h5')

for i in t : 
  arr = tf.keras.preprocessing.image.load_img(i)
  arr = tf.keras.preprocessing.image.img_to_array(arr) / 255
  image = tf.image.resize(arr,[224,224])
  t = model2.predict(np.expand_dims(image,0))
  t1 = model1.predict(np.expand_dims(image,0))
  t3 = model3.predict(np.expand_dims(image,0))

  print(i, np.argmax(t), t[0][np.argmax(t)])
  print(i, np.argmax(t1), t[0][np.argmax(t1)])
  print(i, np.argmax(t3), t[0][np.argmax(t3)])

  print()



/content/benign_3.jpg 0 0.99999714
/content/benign_3.jpg 0 0.99999714
/content/benign_3.jpg 1 2.902074e-06

/content/benign_5.jpg 0 0.9998882
/content/benign_5.jpg 0 0.9998882
/content/benign_5.jpg 1 0.00011174734

/content/melanoma_7.jpg 0 0.99998856
/content/melanoma_7.jpg 0 0.99998856
/content/melanoma_7.jpg 1 1.1412838e-05

/content/melanoma_4.jpg 0 0.999673
/content/melanoma_4.jpg 0 0.999673
/content/melanoma_4.jpg 1 0.0003270189

/content/melanoma.jpeg 0 1.0
/content/melanoma.jpeg 0 1.0
/content/melanoma.jpeg 0 1.0



#Take a photo

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
import matplotlib.image as mpimg
image=mpimg.imread('photo.jpg')